In [1]:
import numpy as np
import tensorflow as tf
import operator
import math

#Las siguientes 2 lineas son para ocupar la GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

from tensorflow.keras.models import Model, load_model
import chess
import chess.pgn

In [2]:
class MCTSNode:
    def __init__(self, game_state, parent=None, move=None):
        self.game_state = game_state
        self.parent = parent
        self.move = move
        self.white=0
        self.black=1
        self.win_counts = {
            self.white: 0,
            self.black: 0,
        }
        self.num_rollouts = 0
        self.children = []
        self.unvisited_moves = list(game_state.legal_moves)
# end::mcts-node[]

# tag::mcts-add-child[]
    def add_random_child(self):
        index = np.random.randint(len(self.unvisited_moves))
        new_move = self.unvisited_moves.pop(index)#Pop elimina el elemento del arreglo
        new_game_state = self.game_state.copy()
        new_game_state.push(new_move)
        new_node = MCTSNode(new_game_state, self, new_move)
        self.children.append(new_node)
        return new_node
# end::mcts-add-child[]

# tag::mcts-record-win[]
    def record_win(self, winner):
        self.win_counts[self.white] += winner[0]
        self.win_counts[self.black] += winner[0]
        self.num_rollouts += 1
# end::mcts-record-win[]

# tag::mcts-readers[]
    def can_add_child(self):
        return len(self.unvisited_moves) > 0

    def is_terminal(self):
        return self.game_state.is_game_over()

    def winning_frac(self, player):
        if player:
            return float(self.win_counts[self.white]) / float(self.num_rollouts)
        else:
            return float(self.win_counts[self.black]) / float(self.num_rollouts)
# end::mcts-readers[]

class MCTSBot:
    def __init__(self, num_rounds, temperature):
        self.num_rounds = num_rounds
        self.temperature = temperature
        pw="white_nn.h5"
        pb="black_nn.h5"
        self.blancas = DeepBot(pw)
        self.negras = DeepBot(pb)
        
# tag::mcts-signature[]
    def select_move(self, game_state):
        root = MCTSNode(game_state)
# end::mcts-signature[]

# tag::mcts-rounds[]
        print("\n")
        for i in range(self.num_rounds):
            print(i,end=" ")
            node = root
            while (not node.can_add_child()) and (not node.is_terminal()):
                node = self.select_child(node)

            # Add a new child node into the tree.
            if node.can_add_child():
                node = node.add_random_child()

            # Simulate a random game from this node.
            winner = self.simulate_random_game(node.game_state.copy())

            # Propagate scores back up the tree.
            while node is not None:
                node.record_win(winner)
                node = node.parent
# end::mcts-rounds[]

        scored_moves = [
            (child.winning_frac(game_state.turn), child.move, child.num_rollouts)
            for child in root.children
        ]
        scored_moves.sort(key=lambda x: x[0], reverse=True)
        for s, m, n in scored_moves[:10]:
            print('%s - %.3f (%d)' % (m, s, n))

# tag::mcts-selection[]
        # Having performed as many MCTS rounds as we have time for, we
        # now pick a move.
        best_move = None
        best_pct = -1.0
        for child in root.children:
            child_pct = child.winning_frac(game_state.turn)
            if child_pct > best_pct:
                best_pct = child_pct
                best_move = child.move
        print('Select move %s with win pct %.3f' % (best_move, best_pct))
        #show_tree(root,max_depth=10)
        return best_move
# end::mcts-selection[]

# tag::mcts-uct[]
    def select_child(self, node):
        """Select a child according to the upper confidence bound for
        trees (UCT) metric.
        """

        #Calcula N(v)
        total_rollouts = sum(child.num_rollouts for child in node.children)
        log_rollouts = math.log(total_rollouts)

        best_score = -1
        best_child = None
        # Loop over each child.
        #Calcula UTC(j)
        for child in node.children:
            # Calculate the UCT score.
            win_percentage = child.winning_frac(node.game_state.turn)
            exploration_factor = math.sqrt(log_rollouts / child.num_rollouts)
            uct_score = win_percentage + self.temperature * exploration_factor
            # Check if this is the largest we've seen so far.
            if uct_score > best_score:
                best_score = uct_score
                best_child = child
        return best_child
    
    def evaluar(self,b):
        mapeo={
            'r':5.63,
            'n':3.05,
            'b':3.33,
            'q':9.5,
            'p':1,
        }
        b=str(b).replace('\n','').replace(' ','').replace('.','').replace('k','').replace('K','')
        white=10
        black=10
        for i in b:
            if i.islower():
                black+=mapeo[i]
            else:
                white+=mapeo[i.lower()]
        return [round(white/(white+black),5),round(black/(white+black),5)]

    def simulate_random_game(self,board):
        board = chess.Board()
        blancas = self.blancas
        negras = self.negras
        while not board.is_game_over():
            move=blancas.select_move(board)[0][0]
            board.push(move)
            if not board.is_game_over():
                move=negras.select_move(board)[0][0]
                board.push(move)
        game.headers["Result"] = board.result()        
        s=board.result()
        if '1/2' in s:
            return self.evaluar(board)
        else:
            return [int(s[0]),int(s[2])]

In [3]:
def codificar(board):
    WHITE=True
    b=str(board).replace(' ','').split('\n')
    a=np.zeros([8,8,5],dtype=np.uint8)
    mapeo={
        '.':[0,0,0,0,0],
        'r':[0,1,0,0,1],
        'n':[0,1,0,1,0],
        'b':[0,1,0,1,1],
        'q':[0,1,1,0,0],
        'k':[0,1,1,0,1],
        'p':[0,1,1,1,0],
        'R':[1,0,0,0,1],
        'N':[1,0,0,1,0],
        'B':[1,0,0,1,1],
        'Q':[1,0,1,0,0],
        'K':[1,0,1,0,1],
        'P':[1,0,1,1,0],
    }
    for i,row in enumerate(b):
        for j,val in enumerate(row):
            if val=='.':
                continue
            a[i,j,:]=mapeo[val]
    return a

In [4]:
#funcion para ordenar un diccionario por su valor de mayor a menor
def order(x):
    return  {k: v for k, v in sorted(x.items(), key=lambda item: item[1], reverse=True)}

In [5]:
def weights(x):
    l=len(x)
    a=(np.arange(l)+1)/l
    return tuple(zip(list(x.keys()),a))
    

In [6]:
class DeepBot:
    def __init__(self,path_model):
        self.eva = load_model(path_model)
        #self.eva.summary()
    
    def select_move(self,board):
        moves=list(board.legal_moves)
        
        if len(moves)>1:
            code_moves=np.zeros([len(moves)*(len(moves)-1),2,8,8,5])
            value=[['xxxxx']*2]*(len(moves)*(len(moves)-1))
            rank={}
            i=0
            for m in moves:
                value[i]=0
                board.push(m)
                b1=codificar(board)
                board.pop()
                for n in moves:
                    if m!=n:
                        board.push(n)
                        b2=codificar(board)
                        board.pop()
                        b=np.array([b1,b2])
                        b=b.reshape([2,8,8,5])
                        code_moves[i,:]=b
                        value[i]=[m,n]
                        i+=1

            predict=self.eva.predict(code_moves)
            if board.turn: #turno de las blancas
                for i,pair in enumerate(value):
                    if predict[i]>0.5: #para las blancas >0.5 significa que el tablero de la izquierda es mejor
                        if pair[0] in rank:
                            rank[pair[0]]+=1
                        else:
                            rank[pair[0]]=1
                    else:
                        if pair[1] in rank:
                            rank[pair[1]]+=1
                        else:
                            rank[pair[1]]=1
            else:
                for i,pair in enumerate(value):
                    if predict[i]>0.5:
                        if pair[1] in rank:
                            rank[pair[1]]+=1
                        else:
                            rank[pair[1]]=1
                    else:
                        if pair[0] in rank:
                            rank[pair[0]]+=1
                        else:
                            rank[pair[0]]=1
            w=weights(order(rank))
        else:
            a=(moves[0],1)
            w=[a]

        return w
        

In [7]:
class RandomBot:
        
    def select_move(self,board):
        moves=list(board.legal_moves)
        return moves[np.random.randint(len(moves))]

In [8]:
pw="white_nn.h5"
board = chess.Board()
blancas = DeepBot(pw)
move=blancas.select_move(board)[0][0]
move

Move.from_uci('c2c4')

In [9]:
board = chess.Board()
blancas = MCTSBot(10, temperature=1.4)
negras = RandomBot()


game = chess.pgn.Game()
game.headers["White"] = "DeepBot"
game.headers["Black"] = "RandomBot"
game.setup(board)  # Not required for the standard
                   # starting position.

node = game


In [10]:
blancas.select_move(board)



0 1 2 3 4 5 6 7 8 9 a2a4 - 1.000 (1)
g2g3 - 1.000 (1)
c2c4 - 1.000 (1)
e2e3 - 1.000 (1)
g2g4 - 1.000 (1)
b1c3 - 1.000 (1)
b2b4 - 1.000 (1)
b2b3 - 1.000 (1)
e2e4 - 1.000 (1)
c2c3 - 1.000 (1)
Select move a2a4 with win pct 1.000


Move.from_uci('a2a4')

In [11]:
board.push(blancas.select_move(board))
board



0 1 2 3 4 5 6 7 8 9 c2c4 - 1.000 (1)
a2a3 - 1.000 (1)
g1h3 - 1.000 (1)
d2d4 - 1.000 (1)
f2f4 - 1.000 (1)
b1c3 - 1.000 (1)
h2h4 - 1.000 (1)
h2h3 - 1.000 (1)
g2g4 - 1.000 (1)
f2f3 - 1.000 (1)
Select move c2c4 with win pct 1.000


TypeError: 'Move' object is not subscriptable

In [ ]:
board.pop()
board

In [ ]:
len(list(board.legal_moves))

In [ ]:
while not board.is_game_over():
    move=blancas.select_move(board)
    board.push(move)
    node = node.add_variation(move) # Add game node
    if not board.is_game_over():
        move=negras.select_move(board)
        board.push(move)
        node = node.add_variation(move) # Add game node
game.headers["Result"] = board.result()        
board.result()


In [ ]:
board
    

In [ ]:
print(game)

In [ ]:
a=['algo']
b=1
c=dict()
c[a[0]]=b
c